In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# 22.  cinvest
# 季度频率。固定资产的变化除以销售，然后取前三个季度该变量的平均值

In [3]:
import pandas as pd
import numpy as np
from utils.diff import diff  # 确保 diff 函数正确导入

# 读取资产负债表
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '固定资产净额'], low_memory=False)

# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

# 读取利润表
df2 = pd.read_csv('季_利润表.csv', usecols=['证券代码', '会计期间', '营业收入'])

# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

# 合并数据
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')

# 计算营业收入的差异
df3 = diff(df3, col=['营业收入'])

# 计算 _cinvest
df3['_cinvest'] = (df3['固定资产净额'] - df3.groupby('证券代码').shift(1)['固定资产净额']) / df3['营业收入'].replace(0, np.nan)

# 计算 cinvest 的三期滚动平均
df3['cinvest'] = df3[['证券代码', '_cinvest']].groupby('证券代码').rolling(window=3).mean().shift(1).reset_index(level=0, drop=True)

# 选择最终列并导出
df4 = df3[['证券代码', '会计期间', 'cinvest']]
df4.to_csv('季_22.csv', index=False, encoding='utf-8-sig')

# 输出结果 DataFrame
print(df4)

          证券代码        会计期间  cinvest
0            1  2000-03-31      NaN
1            1  2000-06-30      NaN
2            1  2000-09-30      NaN
3            1  2000-12-31      NaN
4            1  2001-03-31      NaN
...        ...         ...      ...
571126  920118  2023-09-30      NaN
571127  920118  2023-12-31      NaN
571128  920118  2024-03-31      NaN
571129  920118  2024-06-30      NaN
571130  920118  2024-09-30      NaN

[571131 rows x 3 columns]
